# Introduction to the LangChain Expression Language (LCEL)

中文翻譯英文的 LLM 應用程式

輸入中文, LLM 產生英文翻譯, 然後將翻譯結果印出來


In [8]:
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage
from langchain_core.prompts import PromptTemplate

# Load the environment variables
load_dotenv()

True

# 使用 非 LCEL 的寫法

`PromptTemplate` 的三個常用方法:
- `from_template`: 用來從 template 字串建立 PromptTemplate 物件
- `from_messages`: 用來從 messages list 建立 PromptTemplate 物件
- `fromat()`: 用來格式化 PromptTemplate 物件, 將 template 中的變數替換成實際的值,
  並回傳格式化後的字串
- `invoke()`: 用來執行 PromptTemplate 物件, 回傳 `PromptValue` 物件, 供後續的 Runnable 使用.

In [9]:
from langchain_core.prompts import PromptTemplate
# Create the prompt template
template_str = """
你是一位翻譯專家, 負責將中文翻譯成英文。
你會先將中文順過一次使其流暢，再翻譯成英文。
輸出 JSON 格式的結果, 包含以下三個欄位:
- `input`: 原始的中文
- `revised_input`: 修正過的中文
- `translation`: 翻譯後的英文

<中文>: {input}
"""

# Creat the template object
chat_template = PromptTemplate.from_template(template_str)

# test the template
# promptTemplate.invoke() return PromptValue type.
print(chat_template.invoke({"input": "請問你今天好嗎?"}))

text='\n你是一位翻譯專家, 負責將中文翻譯成英文。\n你會先將中文順過一次使其流暢，再翻譯成英文。\n輸出 JSON 格式的結果, 包含以下三個欄位:\n- `input`: 原始的中文\n- `revised_input`: 修正過的中文\n- `translation`: 翻譯後的英文\n\n<中文>: 請問你今天好嗎?\n'


In [10]:
# Create a chat model
# Prerequest: install langchain-openai package
from langchain.chat_models import init_chat_model

chat_model = init_chat_model("gpt-3.5-turbo", model_provider="openai")

In [11]:
# Create a JSON parser
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser() # return a JSON object



In [12]:
from langchain_core.prompt_values import PromptValue

# Assemble the three components
# They all support the invoke() method

chinese_str = "美國總統特朗普重返白宮屆滿100天，美國的外交、內政以至金融市場都經歷動盪."
prompt:PromptValue = chat_template.invoke({"input": chinese_str})
response = chat_model.invoke(prompt)
json_result = json_parser.invoke(response)

print(json_result)

{'input': '美國總統特朗普重返白宮屆滿100天，美國的外交、內政以至金融市場都經歷動盪.', 'revised_input': '美國總統特朗普重返白宮屆滿100天，美國的外交、內政甚至金融市場都經歷動盪。', 'translation': 'After 100 days since President Trump returned to the White House, the United States has experienced turmoil in its diplomacy, domestic affairs, and even financial markets.'}


# 使用 LCEL 的寫法

In [13]:

# Create the chain
translate_chain = chat_template | chat_model | json_parser

# run the chain
# 因為 chain 中的每個執行步驟都支援 invoke() 方法, 所以可以直接使用 chain.invoke() 方法來執行整個 chain.
# 這樣的寫法會比原本的寫法更簡潔, 而且更容易維護.
translated_result_json = translate_chain.invoke({"input": chinese_str})

print(translated_result_json)

{'input': '美國總統特朗普重返白宮屆滿100天，美國的外交、內政以至金融市場都經歷動盪.', 'revised_input': '美國總統特朗普重返白宮滿100天，美國的外交、內政甚至金融市場都經歷動盪。', 'translation': "After 100 days of President Trump's return to the White House, the United States has experienced turbulence in diplomacy, domestic affairs, and even financial markets."}
